# Испит по Основи на Вештачката Интелигенција

**Датум**: 18.02.2021

## Задача 4

**(25 поени)** 
Разгледуваме игра која се игра со двајца играчи, еден против друг. Правилата на играта се:
- Се игра на табла со димензии `N` по `N`.
- Секој играч има по една кралица.
- Еден потег се состои од два дела:
    - Кралицата се придвижува како што се движи во шахот.
    - Кралицата испалува стрела до поле кое што може да го досегне како кралица во шахот, по што се става X на отстреланото поле.
- Кралицата која нема да може да се придвижи кога ќе дојде на ред, ја губи играта.

Следи интерактивна апликација за оваа игра.

In [1]:
from time import sleep
from plotly import graph_objects as go
import ipywidgets as widgets

In [2]:
class Game:
    def __init__(self, player_1, player_2, N):
        """ Конструктор. """
        self.N = N
        self.starting_position_1, self.starting_position_2 = player_1[2], player_2[2]
        # ова се податоците за играчите
        self.player_1 = {'name': player_1[0], 'type': player_1[1], 'symbol': 'S', 'role': 'MAX'}
        self.player_2 = {'name': player_2[0], 'type': player_2[1], 'symbol': 'H', 'role': 'MIN'}
        self.next_player = self.player_1
        
        self.symbols_fig = {'x': 'x-thin', '·': 'circle-open', 'S': 'star', 'H': 'hexagon'}
        self.scores = {'S': 1, 'H': -1}
        
        self.score = widgets.HTML(description='Статус:', value='')
        self.state = [['·' for y in range(self.N)] for x in range(self.N)]
        self.fig = self.create_fig()
        self.button = widgets.Button(description='Ресетирај')
        self.button.on_click(lambda x: self.reset())
        display(widgets.VBox([widgets.HBox([self.button, self.score]), self.fig]))
        self.reset()
    
    def reset(self):
        """ Ја ресетира играта кога ќе се притисне на копчето `Ресетирај`. """
        self.next_player = self.player_1
        self.update_score('На ред е', self.next_player)
        self.state = [['·' for y in range(self.N)] for x in range(self.N)]
        x, y = self.starting_position_1
        self.state[y][x] = self.player_1['symbol']
        x, y = self.starting_position_2
        self.state[y][x] = self.player_2['symbol']
        self.fig.data[0].marker.symbol = [self.symbols_fig[value] for row in self.state for value in row]
        self.evaluated = {}
        self.take_turns()
    
    def player_took_turn(self, x, y):
        """ Ја обновува таблата за последниот напревен потег на играчот `self.next_player`. """
        x_old, y_old = self.find_symbol(self.state, self.next_player['symbol'])
        self.state[y_old][x_old] = 'x'
        self.state[y][x] = self.next_player['symbol']
        self.fig.data[0].marker.symbol = [self.symbols_fig[value] for row in self.state for value in row]
        self.next_player = self.player_1 if self.next_player == self.player_2 else self.player_2
        winner_symbol = self.check_victory(self.state, self.next_player)
        if winner_symbol != 'keep_playing':
            player_winner = self.player_1 if winner_symbol == self.player_1['symbol'] else self.player_2
            self.update_score('Победник е', player_winner)
            return
        self.update_score('На ред е', self.next_player)
        self.take_turns()
    
    def take_turns(self):
        """ Наизменично ги повикува функциите на играчите, независно дали се човек или вештачка интелигенција. """
        if self.next_player['type'] == 'human':
            return  # wait for player
        else:
            self.score.value += ' -- пресметува'
            state = tuple([tuple(row) for row in self.state])
            result, move = self.minimax(state, self.next_player['role'])
            print(result)
            sleep(1)
            self.player_took_turn(*move)
    
    def validate_move(self, state, position, destination):
        """ Проверува дали е чист патот од `position` до `destination` на таблата `state`. Ако да, тогаш потегот е валиден. """
        x1, y1 = position
        x2, y2 = destination

        y_min, y_max = sorted([y1, y2])
        if x1 == x2:  # иста колона
            for yi in range(y_min+1, y_max):
                if state[yi][x1] != '·':
                    return False # not a valid move
            return True

        x_min, x_max = sorted([x1, x2])
        if y1 == y2:  # иста редица
            for xi in range(x_min+1, x_max):
                if state[y1][xi] != '·':
                    return False # not a valid move
            return True
        
        if abs(x1 - x2) != abs(y1 - y2):  # не се во иста дијагонала
            return False # not a valid move

        for xi, yi in zip(range(x_min+1, x_max), range(y_min+1, y_max)):
            if state[yi][xi] != '·':
                return False # not a valid move
        return True
    
    def update_score(self, message, player):
        """ Ги обновува вредностите на семафорот за кој играч е на ред или дали имаме победник. """
        player_data = ' - '.join(list(player.values())[:3])
        self.score.value = f'{message} <b> {player_data} </b>.'
    
    def update_fig(self, trace, points, selector):
        """ Оваа функција се извршува кога ќе притиснете со глувчето врз графикот. """
        x2, y2 = points.xs[0], points.ys[0]
        x1, y1 = self.find_symbol(self.state, self.next_player['symbol'])
        valid_move = self.validate_move(self.state, (x1, y1), (x2, y2))
        if self.state[y2][x2] == '·' and self.next_player['type'] == 'human' and valid_move:
            self.player_took_turn(x2, y2)

    def create_fig(self):
        """ Ја исцртува таблата. """
        fig = go.FigureWidget()
        x = [x for y in range(self.N) for x in range(self.N)]
        y = [y for y in range(self.N) for x in range(self.N)]
        symbols = [self.symbols_fig[value] for row in self.state for value in row]
        fig.add_scatter(x=x, y=y, mode='markers', marker_size=48, 
                        marker_symbol=symbols, marker_color='LightSkyBlue',
                        marker_line_width=6, marker_line_color='MediumPurple')
        fig.data[0].on_click(self.update_fig)
        fig.update_xaxes(range=[-0.5, self.N - 0.5], dtick=1, title='x', side='top')
        fig.update_yaxes(range=[-0.5, self.N - 0.5], dtick=1, title='y', autorange='reversed')
        fig.update_layout(width=600, height=600, showlegend=False)
        return fig
    
    def minimax(self, node, player, alpha=-2, beta=2, depth=0):
        # вашиот код тука
        return
    
    def find_symbol(self, state, symbol):
        """ Ги пронаоѓа координатите (x, y) на симболот `symbol` во матрицата `state`. """
        return [(x, y) for y, row in enumerate(state) for x, value in enumerate(row) if value == symbol][0]
    
    def expand_state(self, state, player):
        # вашиот код тука
        return
    
    def check_victory(self, state, player):
        """ Го враќа симболот на победникот за матрицата `state`. Следен на потег е играчот `player`. """
        x, y = self.find_symbol(state, player['symbol'])
        for x1, y1 in [(x+1, y), (x-1, y), (x, y+1), (x, y-1), 
                     (x+1, y+1), (x-1, y+1), (x+1, y-1), (x-1, y-1)]:
            if 0 <= x1 < self.N and 0 <= y1 < self.N and state[y1][x1] == '·':
                return 'keep_playing'
        other_player = self.player_2 if player == self.player_1 else self.player_1
        return other_player['symbol']

game = Game(player_1=('Ѕвезда', 'human', (0, 3)), player_2=('Хексагон', 'human', (3, 0)), N=4)

Над овој текст треба да имате интерактивна апликација за играта како на сликата подолу. Прво извршете ги ќелиите, а потоа ако ја немате интерактивната апликација, тогаш нешто не е во ред со вашата инсталација, па побарајте помош од Стефан.

![Тука треба да видите .gif анимација, ама штом го читате ова нешто не е во ред.](images/Кралици.gif)

**Задача:** Напишете вештачка интелигенција која ќе ја игра оваа игра со помош на минимакс алгоритамот, вклучувајќи и алфа-бета поткастрување.

**Совети:** 
- Користете ја задачата Икс-точка од аудиториските вежби за да ја решите оваа задача. Истата е користена како урнек за дефинирање на оваа задача.
- Додавајте код само во функциите `expand_state()` и `minimax()`. Слободно можете да си додавате функции и да ги искористете веќе дадените. Не ви препорачувам менување на останатиот код, но ако сметате дека има потреба - слободно менувајте.